In [2]:
from sklearn.externals import joblib
import pandas as pd
from sklearn.metrics import mean_absolute_error,make_scorer
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Data Preprocessing**

In [6]:
test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test.csv')
test_data = test_data.drop(columns=['id'],axis='column')

cat_features = []
num_features = []

for feature in test_data.columns:
  if 'cat' in feature:
    cat_features.append(feature)
  else:
    num_features.append(feature)

In [8]:
model = joblib.load("/content/drive/My Drive/Colab Notebooks/Final_Model.pkl")

[05:49:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


**FUNCTION 1 (Input single point or array of points)**

In [9]:
def fun1(x_inp):

  unseen_data = pd.DataFrame(data=x_inp,columns=test_data.columns)

  cat_onehot_encode_num_features_test = []

  for col in cat_features:
    vectorizer = joblib.load("/content/drive/My Drive/Colab Notebooks/"+col+".pkl")
    cat_onehot_encode_num_features_test.append(vectorizer.transform(unseen_data[col].values))
  
  for col in num_features:
    if col not in ['loss']:
      cat_onehot_encode_num_features_test.append(unseen_data[col].values.reshape(-1,1))

  X_te = hstack((cat_onehot_encode_num_features_test)).tocsr()

  y_test = model.predict(X_te)

  return y_test


In [22]:
Single_point = test_data.head(1).copy()
fun1(Single_point.to_numpy().tolist())

array([1590.9779], dtype=float32)

In [23]:
array_points = test_data.head(3).copy()
fun1(array_points.to_numpy().tolist())

array([1590.9779, 2136.6711, 9481.576 ], dtype=float32)

**Function 2 (return the metric value)**

In [25]:
def fun2(x_inp,y_target):
  
  unseen_data = pd.DataFrame(data=x_inp,columns=test_data.columns)

  cat_onehot_encode_num_features_test = []

  for col in cat_features:
    vectorizer = joblib.load("/content/drive/My Drive/Colab Notebooks/"+col+".pkl")
    cat_onehot_encode_num_features_test.append(vectorizer.transform(unseen_data[col].values))
  
  for col in num_features:
    if col not in ['loss']:
      cat_onehot_encode_num_features_test.append(unseen_data[col].values.reshape(-1,1))

  X_te = hstack((cat_onehot_encode_num_features_test)).tocsr()

  y_test = model.predict(X_te)
  
  return mean_absolute_error(y_target,y_test)

In [26]:
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train.csv')
train_data = train_data.drop(columns=['id'],axis='column')

X = train_data.drop(columns=['loss'],axis='columns')
y = train_data['loss']

In [32]:
fun2(X.head(3).to_numpy().tolist(),y[0:3:1].values)

624.1319612630208